# 1. Processing Preference Pairs Gathered from M1

Collected data format: 
```json 
{
    "question_id": <int>,
    "question_complete": <str>,
    "course_id": <int>,
    "preference": [
        {
            "A": <str>,
            "B": <str>,
            "overall": <str>,
            "criteria": {
                "overall": <str>,
                "correctness": <str>,
                "relevance": <str>,
                "clarity": <str>,
                "completeness": <str>,
                "other": <str>
            }
        },
        ...
    ]
}
```

Processed data format:
```json
{
    "prompt": "...",
    "chosen": "...", 
    "rejected": "..."
}
```

In [1]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def process_preference_pairs(data):
    processed_data = []
    for d in data:
        prompt = d["question_complete"]
        for p in d["preference"]:
            if p["overall"] == "A":
                chosen_option = "A"
                rejected_option = "B"
            elif p["overall"] == "B":
                chosen_option = "B"
                rejected_option = "A" 
            else:  # skip if the overall preference is not A or B e.g. AB 
                continue
                
            if p[chosen_option] == "..." or p[rejected_option] == "..." or p[chosen_option] == "" or p[rejected_option] == "":
                continue
            processed_data.append({
                "prompt": prompt,
                "chosen": p[chosen_option],
                "rejected": p[rejected_option]
            })
    
    return processed_data


def load_data(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data


def convert_to_jsonl(data, file_path):
    with open(file_path, "w") as f:
        for d in data:
            f.write(json.dumps(d) + "\n")

In [3]:
pp_m1_json_file = "raw_datasets/M1_preference_data_15052024.json"
pp_m1_jsonl_file = "raw_datasets/M1_preference_data_15052024.jsonl"
pp_m1_data = load_data(pp_m1_json_file)

df_pp_m1 = pd.DataFrame(pp_m1_data)
df_pp_m1

,question_id,question_complete,course_id,preference
0,0,Question: Consider the following contains func...,15000,[{'A': 'The asymptotic depth of the contains f...
1,3,Question: What is the asymptotic work of <code...,15000,"[{'A': '...', 'B': '...', 'overall': 'A', 'cri..."
2,4,Question: We have a collection of rectangles i...,15000,[{'A': 'Facts: - Rectangles in the plane have ...
3,5,Question: Which of the following scheduler pol...,15005,[{'A': 'Preemptive scheduling policies allow t...
4,7,"Question: In this week's lecture, you have bee...",15000,"[{'A': 'For the computation g(g(1, x1), g(x2, ..."
...,...,...,...,...
1517,7365,Question: Byzantine consistent broadcast (BCB)...,15003,"[{'A': 'In non-synchronous environments, intro..."
1518,7366,"Question: If process i fails, then eventually ...",15003,"[{'A': 'Yes, the statement is true. If process..."
1519,7368,Question: What happens in the reliable broadca...,15003,[{'A': 'If the completeness property of the fa...
1520,7370,Question: Consider a network that is organized...,15003,"[{'A': 'First, we can use a flooding algorithm..."


In [4]:
# Process the preference pairs and remove duplicates
pp_m1_processed = process_preference_pairs(pp_m1_data)
df_pp_m1_processed = pd.DataFrame(pp_m1_processed)
df_pp_m1_processed.drop_duplicates(subset=["chosen", "rejected"], inplace=True)
pp_m1_processed = df_pp_m1_processed.to_dict(orient="records")

In [10]:
# Split the data into train, test, and validation, with shuffle
seed = 0
train_size = 0.9
val_size = 0.05
test_size = 0.05
pp_m1_train, pp_m1_temp = train_test_split(pp_m1_processed, test_size=(test_size + val_size), random_state=seed)
pp_m1_val, pp_m1_test = train_test_split(pp_m1_temp, test_size=0.5, random_state=seed)

In [11]:
pp_m1_train_file = "datasets/M1_preference_data_15052024_train.jsonl"
pp_m1_val_file = "datasets/M1_preference_data_15052024_val.jsonl"
pp_m1_test_file = "datasets/M1_preference_data_15052024_test.jsonl"

convert_to_jsonl(pp_m1_train, pp_m1_train_file)
convert_to_jsonl(pp_m1_val, pp_m1_val_file)
convert_to_jsonl(pp_m1_test, pp_m1_test_file)

df_pp_m1_train = pd.DataFrame(pp_m1_train)
df_pp_m1_val = pd.DataFrame(pp_m1_val)
df_pp_m1_test = pd.DataFrame(pp_m1_test)

In [12]:
df_pp_m1_train

,prompt,chosen,rejected
0,"Question: A homogeneous, full, vertical wheel,...",Sure! Let's find the velocity of the center of...,Sure! Let's break down the problem step by ste...
1,Question: Does the disparity in class proporti...,The disparity in class proportions does hurt t...,The disparity in class proportions can hurt th...
2,Question: Select all true statements.A penalty...,The true statements are:\n1) The k-NN algorith...,Correct statements:\n1) The k-NN algorithm is ...
3,Question: Consider the (toy) grammar $G$ consi...,In order to cope with simple number agreements...,To address the question of how many rules shou...
4,Question: Why does Intel Itanium contain more ...,Answer: Intel Itanium contains 128 general-pur...,Answer: Intel Itanium contains more general-pu...
...,...,...,...
23946,"Question: In class, we saw Karger's beautiful ...",The main difference between Karger's algorithm...,Karger and Stein modified Karger's algorithm b...
23947,Question: Consider a system of two particles w...,To find the probability of observing $+\frac{\...,"To solve this question, we need to find the st..."
23948,Question: We are given a data set $S=\left\{\l...,\n\nGiven that we are using a nearest neighbor...,Given a data set $S=\left\{\left(\boldsymbol{x...
23949,Question: Recall the Jaccard index that we saw...,**Problem:**\nDesign a locality-sensitive hash...,To design a Locality Sensitive Hashing (LSH) f...


In [13]:
df_pp_m1_val

,prompt,chosen,rejected
0,Question: The exponent $\lambda(21)$ of $\math...,"Let's think step by step. \n\nFirst, we need t...",C: 6. \n\nThe totient function $\lambda(n)$ co...
1,Question: Let $\mathcal C_1$ be a linear code ...,"In coding theory, a linear code is a subspace ...",To determine if $\mathcal C_1 \cup \mathcal C_...
2,Question: Assume you are working on a text edi...,Absolutely! Here is another detailed and relev...,Your colleague may follow these steps to effic...
3,Question: A neutral dielectric in the shape of...,i) To determine the linked charges on the inte...,Step 1: Calculate the linked charge per unit l...
4,"Question: Let $H:\{0,1\}^* \rightarrow \{0,1\}...",The correct option is: $2^{-n}$.\n\nExplanatio...,The correct option is:\n\n$2^{-n}$.\n\nIn cryp...
...,...,...,...
1326,Question: A multiset is an unordered collectio...,To transform a given set `s` to a multiset whe...,To transform a set into a multiset where each ...
1327,Question: Which of the following measures will...,Answer: A) Reducing overheads imposed by the f...,Answer: A) Reducing overheads imposed by the f...
1328,Question: In the following let $\kappa_{1}\lef...,"To show that $\kappa\left(\mathbf{x}, \mathbf{...","To show that $\kappa\left(\mathbf{x}, \mathbf{..."
1329,Question: Let $X$ be a random variable distrib...,Let's analyze this step by step:\n\n1. **Compu...,This is a true statement.\n\nGiven that $X$ is...


In [14]:
df_pp_m1_test

,prompt,chosen,rejected
0,Question: Through a freak accident ($^\copyrig...,To estimate the force between two persons at a...,To estimate the force between two persons at a...
1,"Question: Consider the source $S_1, S_2, \dots...",The source is not stationary.\n\nA stationary ...,The source is not stationary. This can be seen...
2,Question: (Weight initialization) The choice o...,To address the question of weight initializati...,The main topic of the question is whether the ...
3,Question: Given two distributions $P_0$ and $P...,"Alright, let's break this down step by step.\n...",The maximal advantage of a distinguisher utili...
4,"Question: If we have a $n$-bit key, the attack...","To solve this problem, let's break it down int...","To solve this question, we need to understand ..."
...,...,...,...
1326,Question: Let $X$ be a random variable distrib...,The entropy of a discrete random variable $Y$ ...,The entropy of a discrete random variable $W$ ...
1327,Question: Consider an operation we will call s...,"If the function \( f \) is associative, the re...","If the function \( f \) is associative, the re..."
1328,Question: When searching for an entity 𝑒𝑛𝑒𝑤 th...,Answer B can be explained in the following way...,Answer A:\nWhen searching for an entity 𝑒𝑛𝑒𝑤 t...
1329,"Question: Two excellent students, Alice from E...",To design a randomized protocol for Alice and ...,(i) Alice computes the message $m$ of $2$ bits...


# 2. Process other datasets

## Stanford Human Preferences Dataset (SHP)

Link: [https://huggingface.co/datasets/stanfordnlp/SHP](https://huggingface.co/datasets/stanfordnlp/SHP)

- `post_id`: the ID of the Reddit post (string)
- `domain`: the subreddit and split the example is drawn from, separated by an underscore (string)
- `upvote_ratio`: the percent of votes received by the post that were positive (aka upvotes) (float)
- `history`: the post title concatented to the post body (string)
- `c_root_id_A`: the ID of comment A (string)
- `c_root_id_B`: the ID of comment B (string)
- `created_at_utc_A`: utc timestamp of when comment A was created (integer)
- `created_at_utc_B`: utc timestamp of when comment B was created (integer)
- `score_A`: (# positive votes - # negative votes + 1) received by comment A (integer)
- `score_B`: (# positive votes - # negative votes + 1) received by comment B (integer)
- `human_ref_A`: text of comment A (string)
- `human_ref_B`: text of comment B (string)
- `labels`: the preference label -- **it is 1 if A is preferred to B; 0 if B is preferred to A**. This was randomized such that the label distribution is roughly 50/50. (integer)
- `seconds_difference`: how many seconds after the less preferred comment the more preferred one was created (will always be >= 0) (integer)
- `score_ratio`: the ratio of the more preferred comment's score to the less preferred comment's score (will be >= 1) (float)

In [16]:
from datasets import load_dataset

In [ ]:
data_askengineers = load_dataset("stanfordnlp/shp", data_dir="askengineers")
data_askphysics = load_dataset("stanfordnlp/shp", data_dir="askphysics")
data_askscience = load_dataset("stanfordnlp/shp", data_dir="askscience")
data_explainlikeimfive = load_dataset("stanfordnlp/shp", data_dir="explainlikeimfive")

In [ ]:
train_datasets = {
    "askengineers": data_askengineers["train"],
    "askphysics": data_askphysics["train"],
    "askscience": data_askscience["train"],
    "explainlikeimfive": data_explainlikeimfive["train"]
}

test_datasets = {
    "askengineers": data_askengineers["test"],
    "askphysics": data_askphysics["test"],
    "askscience": data_askscience["test"],
    "explainlikeimfive": data_explainlikeimfive["test"]
}

val_datasets = {
    "askengineers": data_askengineers["validation"],
    "askphysics": data_askphysics["validation"],
    "askscience": data_askscience["validation"],
    "explainlikeimfive": data_explainlikeimfive["validation"]
}

In [ ]:
# Print infos about the datasets
print("Infos for train datasets")
for k, v in train_datasets.items():
    print(f"Number of samples in {k}: {len(v)}")
    
print("\nInfos for test datasets")
for k, v in test_datasets.items():
    print(f"Number of samples in {k}: {len(v)}")
    
print("\nInfos for validation datasets")
for k, v in val_datasets.items():
    print(f"Number of samples in {k}: {len(v)}")

In [ ]:
# Example of a sample
train_datasets["askengineers"][0]

In [ ]:
def process_shp_sub_dataset(dataset):
    processed_data = []
    for d in dataset:
        prompt = d["history"]
        if d["labels"] == 1:
            chosen_option = "human_ref_A"
            rejected_option = "human_ref_B"
        elif d["labels"] == 0:
            chosen_option = "human_ref_B"
            rejected_option = "human_ref_A"
        else:  # skip if the overall preference is not A or B e.g. AB 
            continue
            
        processed_data.append({
            "prompt": prompt,
            "chosen": d[chosen_option],
            "rejected": d[rejected_option]
        })
    return processed_data

In [ ]:
processed_training_data = {
    "askengineers": process_shp_sub_dataset(train_datasets["askengineers"]),
    "askphysics": process_shp_sub_dataset(train_datasets["askphysics"]),
    "askscience": process_shp_sub_dataset(train_datasets["askscience"]),
    "explainlikeimfive": process_shp_sub_dataset(train_datasets["explainlikeimfive"])
}

processed_test_data = {
    "askengineers": process_shp_sub_dataset(test_datasets["askengineers"]),
    "askphysics": process_shp_sub_dataset(test_datasets["askphysics"]),
    "askscience": process_shp_sub_dataset(test_datasets["askscience"]),
    "explainlikeimfive": process_shp_sub_dataset(test_datasets["explainlikeimfive"])
}

processed_val_data = {
    "askengineers": process_shp_sub_dataset(val_datasets["askengineers"]),
    "askphysics": process_shp_sub_dataset(val_datasets["askphysics"]),
    "askscience": process_shp_sub_dataset(val_datasets["askscience"]),
    "explainlikeimfive": process_shp_sub_dataset(val_datasets["explainlikeimfive"])
}

In [ ]:
processed_test_data["askscience"][0]

In [ ]:
print("Processed training data")
for k, v in processed_training_data.items():
    print(f"Number of samples in {k}: {len(v)}")
    
print("\nProcessed test data")
for k, v in processed_test_data.items():
    print(f"Number of samples in {k}: {len(v)}")
    
print("\nProcessed validation data")
for k, v in processed_val_data.items():
    print(f"Number of samples in {k}: {len(v)}")

In [ ]:
# Convert to jsonl
training_paths = {
    "askengineers": "datasets/askengineers_train.jsonl",
    "askphysics": "datasets/askphysics_train.jsonl",
    "askscience": "datasets/askscience_train.jsonl",
    "explainlikeimfive": "datasets/explainlikeimfive_train.jsonl"
}

test_paths = {
    "askengineers": "datasets/askengineers_test.jsonl",
    "askphysics": "datasets/askphysics_test.jsonl",
    "askscience": "datasets/askscience_test.jsonl",
    "explainlikeimfive": "datasets/explainlikeimfive_test.jsonl"
}

val_paths = {
    "askengineers": "datasets/askengineers_val.jsonl",
    "askphysics": "datasets/askphysics_val.jsonl",
    "askscience": "datasets/askscience_val.jsonl",
    "explainlikeimfive": "datasets/explainlikeimfive_val.jsonl"
}

In [ ]:
for k, v in processed_training_data.items():
    convert_to_jsonl(v, training_paths[k])
    
for k, v in processed_test_data.items():
    convert_to_jsonl(v, test_paths[k])
    
for k, v in processed_val_data.items():
    convert_to_jsonl(v, val_paths[k])

## orpo-dpo-mix-40k

Link: [https://huggingface.co/datasets/mlabonne/orpo-dpo-mix-40k?row=1](https://huggingface.co/datasets/mlabonne/orpo-dpo-mix-40k?row=1) 

In [42]:
orpo_dpo_mix = load_dataset("mlabonne/orpo-dpo-mix-40k")
orpo_dpo_mix = orpo_dpo_mix.filter(lambda r: r["source"] != "toxic-dpo-v0.2")

In [43]:
orpo_dpo_mix = orpo_dpo_mix["train"]

In [44]:
orpo_dpo_mix[0]["chosen"][1]["content"]

"As you step onto the teleportation platform, there's a momentary sense of disorientation before your surroundings change abruptly. You find yourself standing on the outskirts of Zephyria, gazing at the sprawling metropolis that glows softly under the starlit canvas above. A gentle breeze, carrying hints of exotic fragrances from unknown flora, greets you. \n\nYou begin to walk along the radiant pathway, each cobblestone pulsating beneath your feet, resonating with a rhythm that seems almost alive. The cityscape ahead shimmers with countless shades of sapphire, amethyst, and turquoise, their reflections dancing upon the glassy surfaces of the spiraling towers around you. The air vibrates subtly with an underlying hum, a symphony of unseen energy sources powering this celestial city.\n\nApproaching one of the spiraling edifices, you reach out to touch its surface. It feels unexpectedly warm, humming slightly under your fingertips. Its translucent walls ripple with colors, revealing glim

In [45]:
def process_orpo_dpo_mix(dataset):
    processed_data = []
    for d in dataset:
        processed_data.append({
            "prompt": d["prompt"],
            "chosen": d["chosen"][1]["content"],
            "rejected": d["rejected"][1]["content"]
        })
    return processed_data

In [46]:
processed_orpo_dpo_mix = process_orpo_dpo_mix(orpo_dpo_mix)
orpo_dpo_train, orpo_dpo_temp = train_test_split(processed_orpo_dpo_mix, test_size=(test_size + val_size), random_state=seed)
orpo_dpo_val, orpo_dpo_test = train_test_split(orpo_dpo_temp, test_size=0.5, random_state=seed)

orpo_dpo_train_file = "datasets/orpo_dpo_mix_train.jsonl"
orpo_dpo_val_file = "datasets/orpo_dpo_mix_val.jsonl"
orpo_dpo_test_file = "datasets/orpo_dpo_mix_test.jsonl"

convert_to_jsonl(orpo_dpo_train, orpo_dpo_train_file)
convert_to_jsonl(orpo_dpo_val, orpo_dpo_val_file)
convert_to_jsonl(orpo_dpo_test, orpo_dpo_test_file)

In [47]:
print("Number of samples in orpo_dpo_train:", len(orpo_dpo_train))
print("Number of samples in orpo_dpo_val:", len(orpo_dpo_val))
print("Number of samples in orpo_dpo_test:", len(orpo_dpo_test))

Number of samples in orpo_dpo_train: 39333
Number of samples in orpo_dpo_val: 2185
Number of samples in orpo_dpo_test: 2186
